This notebook aims to test different registration methods using the Ground Truth points that were manually detected on EM and LM images.

#### Loading the Ground truth files in XML

In [3]:
from utils import xml_to_dataframe


Load the locations for fiducial particles in EM and LM. The locations were manually detected using ICY software and were saved in XML format. These point locations are considered to be the ground truth. For further processing we need to load the XML format and convert it into Pandas dataframe.

In [ ]:
target_xml = 'Ground_truth_fiducials_EM.xml'
target_df = xml_to_dataframe(target_xml)
print(target_df)

source_xml = 'Ground_truth_fiducials_LM.xml'
source_df = xml_to_dataframe(source_xml)
print(source_df)